In [1]:
import pandas as pd
import numpy as np
import random
import os
from sklearn.preprocessing import OneHotEncoder
from neural_network_wrapper import NeuralNetworkWrapper
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
import optimizers

import json

In [2]:
data_simple_train_100 = pd.read_csv("./projekt1/classification/data.simple.train.100.csv")
data_simple_train_500 = pd.read_csv("./projekt1/classification/data.simple.train.500.csv")
data_simple_train_1000 = pd.read_csv("./projekt1/classification/data.simple.train.1000.csv")
data_simple_train_10000 = pd.read_csv("./projekt1/classification/data.simple.train.10000.csv")

data_simple_test_100 = pd.read_csv("./projekt1/classification/data.simple.test.100.csv")
data_simple_test_500 = pd.read_csv("./projekt1/classification/data.simple.test.500.csv")
data_simple_test_1000 = pd.read_csv("./projekt1/classification/data.simple.test.1000.csv")
data_simple_test_10000 = pd.read_csv("./projekt1/classification/data.simple.test.10000.csv")

In [3]:
def prepare_data_simple(train_data, test_data):
    
    X_train = np.array(train_data.loc[:, ['x', 'y']])
    y_train = train_data.cls
    y_train -= 1
    #one hot encoding
    y_ohc = np.zeros((y_train.size, int(np.max(y_train))+1))
    y_ohc[np.arange(y_train.size),y_train.astype(np.int)] = 1
    y_train = y_ohc
    
    X_test = np.array(test_data.loc[:, ['x', 'y']])
    y_test = test_data.cls
    y_test -= 1
    #one hot encoding
    y_ohc = np.zeros((y_test.size, int(np.max(y_test))+1))
    y_ohc[np.arange(y_test.size),y_test.astype(np.int)] = 1
    y_test = y_ohc
    
    
    # Are we supposed to use StandardScaler?
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    
    return {"X_train": X_train,
           "X_test": X_test,
           "y_train": y_train,
           "y_test": y_test}

In [4]:
data = [{"dataset name": "Data simple 100 obs",
        "data": prepare_data_simple(data_simple_train_100, data_simple_test_100)},
       {"dataset name": "Data simple 500 obs",
        "data": prepare_data_simple(data_simple_train_500, data_simple_test_500)},
       {"dataset name": "Data simple 1000 obs",
        "data": prepare_data_simple(data_simple_train_1000, data_simple_test_1000)},
       {"dataset name": "Data simple 10000 obs",
        "data": prepare_data_simple(data_simple_train_10000, data_simple_test_10000)}]

In [5]:
input_dim = 2
neuron_numbers = [4, 4, 2]
output_activation = ['sigmoid']
seed=42 #reproducibility
num_epochs=5
loss_function = 'logistic_loss'
learning_rates = {"LR = 0.1" : 0.1,
                 "LR = 0.01" : 0.01,
                 "LR = 0.001" : 0.001}
                  


# Things we would like to test
activation_functions = ['relu', 'leaky_relu']
batch_size = 1#[2*(i+1) for i in range(5)]
optimizer_beta = [0, 0.5, 0.9] # 0 means no inertia


experiment_dict = {
    "input_dim" : input_dim,
    "neuron_numbers" : neuron_numbers, # number of neurons in consecutive layers
    "activation_functions" : activation_functions,
    "loss_function" : loss_function,
    "learning_rates" : learning_rates,
    "batch_size" : batch_size,
    "num_epochs" : num_epochs,
    "seed" : seed,
    "output_activation" : output_activation
}

In [6]:
def perform_experiment(dataset,
                       d,
                       exp_objective,
                       exp_values,
                       num_reps):
    """
    """
    X_train = dataset['X_train']
    y_train = dataset['y_train']
    X_test = dataset['X_test']
    y_test = dataset['y_test']

    d = d.copy()
    
    for k in exp_values.keys():
        d[k] = {}
        d[k]['test_accuracy'] = []
    
    for i in range(num_reps):
        print(f"Experiment {i+1}/{num_reps}")
    
        # reproducibility issues
        random.seed(d['seed'] + i)
        np.random.seed(d['seed'] + i)
    
        # testing learning rate
        if exp_objective == 'lr':
            for k, v in exp_values.items():
            
                

                NN = NeuralNetworkWrapper(d['input_dim'],
                                      d['neuron_numbers'],
                                      ['relu'] * (len(d['neuron_numbers']) - 1) + d['output_activation'],
                                      d['loss_function'],
                                      v,
                                      optimizers.Optimizer(),
                                      d['batch_size'])
                
                NN.train(X_train,
                        y_train,
                        d['num_epochs'],
                        validation_split = 0,
                        test_accuracy=(X_test, y_test),
                        verbosity=False)
                
                d[k]['test_accuracy'].append(NN.test_accuracy)

    for k in exp_values.keys():
        # aggregating results
        d[k]['test_accuracy_mean'] = np.mean(np.array(d[k]['test_accuracy']).T, axis=1)
        d[k]['test_accuracy_std'] = np.std(np.array(d[k]['test_accuracy']).T, axis=1)

        d[k] = {"Accuracy": d[k]['test_accuracy_mean'],
               "Accuracy std": d[k]['test_accuracy_std'],
               "Best Accuracy": np.max(d[k]['test_accuracy_mean']),
               "Best Accuracy std": d[k]['test_accuracy_std'][np.argmax(d[k]['test_accuracy_mean'])]}

    return {k: d[k] for k in exp_values.keys()}


In [7]:
# perform_experiment(data[0]['data'],
#                    experiment_dict,
#                    'lr',
#                    {'lr=0.1' : 0.001,
#                    'lr=0.2' : 0.002},
#                   10)

In [8]:
def experiments_pipeline(data,
                         experiment_dict,
                         num_reps=1,
                         save_to_file=False):
    """
    """
    d = experiment_dict.copy()
    output = {}
    # Experiments for each dataset
    for dataset in data:
        output[dataset['dataset name']] = perform_experiment(dataset['data'],
                                                     experiment_dict,
                                                     'lr',
                                                     {'lr=0.1' : 0.001,
                                                      'lr=0.2' : 0.002},
                                                     num_reps)
    return output

In [9]:
ans = experiments_pipeline(data, experiment_dict, num_reps=5)

Experiment 1/5
Final loss: 1.343
Final loss: 0.641
Experiment 2/5
Final loss: 0.941
Final loss: 1.125
Experiment 3/5
Final loss: 1.797
Final loss: 0.801
Experiment 4/5
Final loss: 1.330
Final loss: 1.010
Experiment 5/5
Final loss: 1.862
Final loss: 1.242
Experiment 1/5
Final loss: 0.929
Final loss: 0.239
Experiment 2/5
Final loss: 0.302
Final loss: 0.414
Experiment 3/5
Final loss: 1.419
Final loss: 0.177
Experiment 4/5
Final loss: 0.903
Final loss: 0.309
Experiment 5/5
Final loss: 0.527
Final loss: 0.248
Experiment 1/5
Final loss: 0.569
Final loss: 0.120
Experiment 2/5
Final loss: 0.164
Final loss: 0.148
Experiment 3/5
Final loss: 1.149
Final loss: 0.087
Experiment 4/5
Final loss: 0.512
Final loss: 0.136
Experiment 5/5
Final loss: 0.335
Final loss: 0.123
Experiment 1/5
Final loss: 0.042


/home/jakubkala/IAD/semestr-2/mgu/deep-learning-methods/loss_functions.py:5: RuntimeWarning: divide by zero encountered in log
  loss = (-1 / m) * (np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat)))
/home/jakubkala/IAD/semestr-2/mgu/deep-learning-methods/loss_functions.py:5: RuntimeWarning: invalid value encountered in multiply
  loss = (-1 / m) * (np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat)))
/home/jakubkala/IAD/semestr-2/mgu/deep-learning-methods/loss_functions.py:11: RuntimeWarning: invalid value encountered in true_divide
  return np.divide(1 - y, 1 - y_hat) - np.divide(y, y_hat)
/home/jakubkala/IAD/semestr-2/mgu/deep-learning-methods/activation_functions.py:21: RuntimeWarning: invalid value encountered in greater
  derivative[Z > 0] = 1


Final loss: nan
Experiment 2/5
Final loss: 0.037
Final loss: nan
Experiment 3/5
Final loss: nan
Final loss: nan
Experiment 4/5
Final loss: 0.075
Final loss: nan
Experiment 5/5
Final loss: nan
Final loss: nan


In [10]:
ans

{'Data simple 100 obs': {'lr=0.1': {'Accuracy': array([0.53 , 0.554, 0.562, 0.59 , 0.644]),
   'Accuracy std': array([0.07823043, 0.08979978, 0.09846827, 0.1154123 , 0.15994999]),
   'Best Accuracy': 0.6439999999999999,
   'Best Accuracy std': 0.15994999218505765},
  'lr=0.2': {'Accuracy': array([0.604, 0.686, 0.73 , 0.76 , 0.812]),
   'Accuracy std': array([0.10442222, 0.11808472, 0.14042792, 0.13696715, 0.07547185]),
   'Best Accuracy': 0.812,
   'Best Accuracy std': 0.07547184905645281}},
 'Data simple 500 obs': {'lr=0.1': {'Accuracy': array([0.5976, 0.7556, 0.8044, 0.8272, 0.842 ]),
   'Accuracy std': array([0.11837838, 0.140692  , 0.15813614, 0.17005223, 0.176     ]),
   'Best Accuracy': 0.842,
   'Best Accuracy std': 0.176},
  'lr=0.2': {'Accuracy': array([0.8228, 0.9   , 0.942 , 0.9712, 0.9772]),
   'Accuracy std': array([0.06961149, 0.02356268, 0.01507315, 0.02278947, 0.01916664]),
   'Best Accuracy': 0.9772000000000001,
   'Best Accuracy std': 0.01916663768113752}},
 'Data sim

In [11]:
import matplotlib.pyplot as plt

In [12]:

# fig, a = plt.subplots(2, 2)

# #plt.figure(figsize=(21,12))

# a[0][0].plot([i+1 for i in range(d['num_epochs'])], d['train_loss_mean'], label = d['experiment_name'])
# a[0][1].plot([i+1 for i in range(d['num_epochs'])], d['valid_loss_mean'], label = d['experiment_name'])
# a[1][0].plot([i+1 for i in range(d['num_epochs'])], d['train_accuracy_mean'], label = d['experiment_name'])
# a[1][1].plot([i+1 for i in range(d['num_epochs'])], d['valid_accuracy_mean'], label = d['experiment_name'])

# a[0][0].plot([i+1 for i in range(d1['num_epochs'])], d1['train_loss_mean'], label = d1['experiment_name'])
# a[0][1].plot([i+1 for i in range(d1['num_epochs'])], d1['valid_loss_mean'], label = d1['experiment_name'])
# a[1][0].plot([i+1 for i in range(d1['num_epochs'])], d1['train_accuracy_mean'], label = d1['experiment_name'])
# a[1][1].plot([i+1 for i in range(d1['num_epochs'])], d1['valid_accuracy_mean'], label = d1['experiment_name'])

# a[0][1].legend(loc="upper right")

# a[0][0].title.set_text('Train Loss')
# a[0][1].title.set_text('Validation Loss')
# a[1][0].title.set_text('Train Accuracy')
# a[1][1].title.set_text('Validation Accuracy')

# fig.set_figheight(9)
# fig.set_figwidth(15)

# plt.show()